In [ ]:
### OECD FDI: FOREIGN DIRECT INVESTMENT

In [ ]:
### RUN EVERY TIME: INITIALIZATION

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', -1) ### To display long strings
import math
import requests
import json ### To correct JSON structure before unpacking
import xml.etree.ElementTree as et
import gc
import os
import datetime
import time
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
import seaborn as sns
#%load_ext line_profiler

In [ ]:
### RUN EVERY TIME: VERSION CONTROL

from platform import python_version
print('pandas version: ', pd.__version__)
print('python version: ', python_version())

In [4]:
### RUN EVERY TIME: MAIN CONSTANTS

### MultiIndex level slice constant:
All = slice(None)
### Universe path:
str_path_universe = 'Data_Files/Source_Files/acadian_universe.xlsx'
### OECD FDI datasets:
str_path_oecd_fdi_dataset = 'Data_Files/Source_Files/oecd_dataset.h5'
str_key_do_total_oecd_fdi_dataset = 'fdi_total_outward_dataset'
str_key_di_total_oecd_fdi_dataset = 'fdi_total_inward_dataset'
str_key_do_equity_oecd_fdi_dataset = 'fdi_equity_outward_dataset'
str_key_di_equity_oecd_fdi_dataset = 'fdi_equity_inward_dataset'
str_path_oecd_fdi_augmented = 'Data_Files/Source_Files/oecd_augmented.h5'
str_key_do_total_oecd_fdi_augmented = 'fdi_total_outward_augmented'
str_key_do_equity_oecd_fdi_augmented = 'fdi_equity_outward_augmented'
str_path_oecd_fdi_options = 'Data_Files/Source_Files/oecd_options.h5'
str_key_total_oecd_fdi_options = 'fdi_total_outward_options'
str_key_equity_oecd_fdi_options = 'fdi_equity_outward_options'
### Technical Constants:
str_date_end = '2022-10-31'
date_start = pd.Timestamp('1989-12-29')
date_end = pd.Timestamp(str_date_end)
date_ison = pd.Timestamp('1994-12-31')

In [5]:
### DEFINING COUNTRY CODES EXTRACTOR

def get_country_codes(use_local_copy = False):  
    ### In case if URL is unavailable:
    if (use_local_copy):
        url_country_code = 'Data_Files/Source_Files/countrycode.html'
    ### Online extraction:
    else:
        url_country_code = 'https://countrycode.org/'
    df_full_codes = pd.read_html(url_country_code, index_col = 'COUNTRY')[0]
    df_full_codes[['ISO SHORT', 'ISO LONG']] = df_full_codes['ISO CODES'].str.split(' / ', expand = True)
    df_result = df_full_codes[['ISO SHORT', 'ISO LONG']].sort_index()    
    df_result.index = df_result.index.str.upper()
    ### Results output:
    return df_result

In [6]:
### DEFINING EXTRACTION UNIVERSE DATA FROM MS EXCEL SOURCE (TO BE IGNORED IN PRODUCT CODE)

def ison_membership_converting(str_path_universe, date_end, bool_daily = False, int_backfill_months = 0):
    ### Defining business-month-end reindexation on country level:
    def country_modify(ser_raw_country, date_end):
        ser_res_country = ser_raw_country.droplevel(0).resample('MS').last().resample('BM').last()
        range_country = pd.date_range(ser_res_country.index[0], date_end, freq = 'BM')
        return ser_res_country.reindex(range_country).ffill()
    ### Markets encoding table:
    dict_markets = {50 : 'DM', 57 : 'EM', 504 : 'FM', 0: np.NaN}     
    ### Loading source file:
    df_raw_universe = pd.read_excel(engine = 'openpyxl', io = str_path_universe, sheet_name = 'Switchers', header = 0, parse_dates = True, index_col = [0, 1],
                                 na_values = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', 
                                             '1.#QNAN', 'N/A', 'NULL', 'NaN', 'n/a', 'nan', 'null'], keep_default_na = False)
    ### Converting source file:
    df_raw_universe.index.names = ['Country', 'Date']
    ser_raw_universe = df_raw_universe['Region']
    ser_raw_universe.fillna(0, inplace = True)
    ser_raw_universe.name = 'Market'
    ### By country reindexation and translation:
    ser_res_universe = ser_raw_universe.groupby('Country').apply(country_modify, date_end)
    ser_res_universe.index.names = ['Country', 'Date']
    ser_res_universe = ser_res_universe.replace(dict_markets).reorder_levels([1, 0]).sort_index() 
    ### Expanding membership for primary regions members by backfilling:
    if int_backfill_months:
        ### List of regions:
        list_region = list(ser_res_universe.dropna().unique())
        ### Initialising of collection of series with backfilled data for each region:
        list_ison_backfill = []
        ### Regions looping:
        for iter_region in list_region:
            ### Defining start of region date:
            date_first_valid = ser_res_universe.loc[ser_res_universe == iter_region].first_valid_index()[0]
            ### Creating dates index to backfilling:
            idx_date_backfill = pd.date_range(end = date_first_valid, periods = int_backfill_months + 1, freq = 'BM')[: -1]
            ### Creating primary countries index to backfilling:            
            idx_region_backfill = ser_res_universe.loc[ser_res_universe == iter_region].loc[date_first_valid, All].index.get_level_values('Country')
            ### Creating full index:
            idx_ison_backfill = pd.MultiIndex.from_product([idx_date_backfill, idx_region_backfill])
            ### Series with backfilled data:
            list_ison_backfill.append(pd.Series(iter_region, index = idx_ison_backfill))
        ### Combination of backfilled series and original ISON data:    
        ser_res_universe = ser_res_universe.combine_first(pd.concat(list_ison_backfill, axis = 0)).sort_index()  
        ser_res_universe.index.names = ['Date', 'Country']
    ### Converting to daily frequency:
    if bool_daily:
        ser_res_universe = ser_res_universe.reset_index('Country').groupby('Country').resample('B').ffill()['Market'].swaplevel().sort_index()    
    ### Results output:
    ser_res_universe.name = 'Market'
    return ser_res_universe

In [7]:
### RUN EVERY TIME: COMMON DATA EXTRACTION STEPS

### World Country Codes:
df_country_codes = get_country_codes()
### ISON membership history:
ser_ison_membership = ison_membership_converting(str_path_universe, pd.to_datetime(str_date_end))
### ISON LONG IDs list:
list_ison_long = list(df_country_codes.loc[df_country_codes['ISO SHORT'].isin(ser_ison_membership.index.get_level_values('Country').unique()), 'ISO LONG'].values)
### ISON current status:
ser_ison_status = ser_ison_membership.loc[str_date_end].droplevel('Date')
### ISON stats:
int_ison_number = len(list_ison_long)
list_regions = ['DM', 'EM', 'FM']
dict_ison_len = {}
dict_ison_len['Full Universe'] = int_ison_number
for iter_region in list_regions:
    dict_ison_len[iter_region] = len(ser_ison_status[ser_ison_status == iter_region])
ser_market_len = pd.Series(dict_ison_len)
ser_market_len.index.names = ['Market']    

In [8]:
### OECD FDI: GENERAL DATA PREPARATION

### Constants:
All = slice(None)
str_oecd_base_url = 'https://stats.oecd.org/sdmx-json/data/'
str_oecd_structure_url = 'https://stats.oecd.org/restsdmx/sdmx.ashx/GetDataStructure/'
str_fdi_pos_dataset_add = 'FDI_POS_CTRY'

In [9]:
### OECD FDI: REQUESTS SESSION INITIALIZING

request_session = requests.Session()
### For avoiding data request errors:
dict_header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'}
request_session.headers.update(dict_header)

In [10]:
### OECD FDI: FDI POSITION STRUCTURE REQUEST

obj_oecd_structure = request_session.get(str_oecd_structure_url + str_fdi_pos_dataset_add)
xml_tree_root = et.fromstring(obj_oecd_structure.content)
dict_concepts = {}
dict_dimensions = {}
dict_codelists = {}
for xml_tree_child in xml_tree_root:
    if xml_tree_child.tag.endswith('Concepts'):
        for xml_tree_grand in xml_tree_child:
            str_concept_id = xml_tree_grand.attrib['id']
            str_concept_name = xml_tree_grand[0].text
            dict_concepts[str_concept_id] = str_concept_name
    if xml_tree_child.tag.endswith('KeyFamilies'):
        for xml_tree_family in xml_tree_child:
            for xml_tree_component in xml_tree_family:
                if xml_tree_component.tag.endswith('Components'):
                    for xml_tree_measure in xml_tree_component:
                        if xml_tree_measure.tag.endswith('Dimension'):
                            str_concept_id = xml_tree_measure.attrib['conceptRef']
                            str_concept_cl_id = xml_tree_measure.attrib['codelist']
                            dict_dimensions[str_concept_id] = str_concept_cl_id
    if xml_tree_child.tag.endswith('CodeLists'):       
        for num_tree_grand, xml_tree_grand in enumerate(xml_tree_child):
            str_codelist_id = xml_tree_grand.attrib['id']
            dict_codelist = {}
            for xml_tree_codelist in xml_tree_grand:                
                if xml_tree_codelist.tag.endswith('Code'):
                    str_code_id = xml_tree_codelist.attrib['value']
                    str_code_value = xml_tree_codelist[0].text
                    dict_codelist[str_code_id] = str_code_value
            dict_codelists[str_codelist_id] = dict_codelist

In [11]:
### OECD FDI: DIMENSIONS

pd.concat([pd.Series(dict_concepts), pd.Series(dict_dimensions)], axis = 1, sort = False).dropna()

,0,1
COU,Reporting country,CL_FDI_POS_CTRY_COU
MEASURE,Currency,CL_FDI_POS_CTRY_MEASURE
MEASURE_PRINCIPLE,Measurement principle,CL_FDI_POS_CTRY_MEASURE_PRINCIPLE
FDI_TYPE,Type of FDI,CL_FDI_POS_CTRY_FDI_TYPE
TYPE_ENTITY,Type of entity,CL_FDI_POS_CTRY_TYPE_ENTITY
ACCOUNTING_ENTRY,Accounting entry,CL_FDI_POS_CTRY_ACCOUNTING_ENTRY
LEVEL_COUNTERPART,Level of counterpart,CL_FDI_POS_CTRY_LEVEL_COUNTERPART
COUNTERPART_AREA,Partner country/territory,CL_FDI_POS_CTRY_COUNTERPART_AREA
TIME,Year,CL_FDI_POS_CTRY_TIME


In [12]:
### OECD FDI: FDI POSITION CONCEPT SOURCE CODELISTS:

dict_codelists['CL_FDI_POS_CTRY_FDI_TYPE']

{'LE_FA_F': 'FDI positions -Total',
 'LE_FA_F5': 'FDI positions - Equity (including reinvestment of earnings)',
 'LE_FA_FL': 'FDI positions - Debt'}

In [13]:
### OECD FDI: FDI TOTAL POSITION PARAMETERS PREPARATION:

### Currency:
str_measure = 'USD'
### Direction:
str_direction = '+'.join(['DI', 'DO'])
### Investment type:
str_fdi_type = '+'.join(['LE_FA_F', 'LE_FA_F5'])
### Residence defining:
str_residence = 'ALL'
### Accounting way:
str_accounting = 'NET' # '+'.join(['NET', 'A', 'L']) # 
### Level counterpart(???):
str_counterpart = 'IMC'

In [14]:
### OECD FDI: FDI POSITION PARAMETERS PREPARATION:

### ISON Countries collecting:
df_ison_countries = df_country_codes.set_index('ISO SHORT', append = True).reset_index('COUNTRY', drop = True)
df_ison_countries = df_ison_countries.reindex(ser_ison_membership.index.get_level_values(1).unique().to_list())
ser_ison_countries = df_ison_countries.reset_index().set_index('ISO LONG').squeeze()
### OECD reporters vs ISON members:
ser_oecd_reporters = pd.Series(dict_codelists['CL_FDI_POS_CTRY_COU'])
ser_oecd_reporters = ser_oecd_reporters.to_frame().join(ser_ison_countries).drop(0, axis = 1).squeeze()
for iter_iso_long in (ser_oecd_reporters[ser_oecd_reporters.isna()].index.get_level_values(0)):
    if iter_iso_long in ser_ison_countries.index:
        print('OECD Reporter country with no ISON match:', iter_iso_long)
### ISON countries with no OECD reporter match:
set_no_reporters = set(ser_ison_countries.dropna().index) - set(ser_oecd_reporters.index)
print('ISON countries with no OECD reporter match:', set_no_reporters, '(', len(set_no_reporters), ')')           
### OECD partners vs ISON members:
ser_oecd_partners = pd.Series(dict_codelists['CL_FDI_POS_CTRY_COUNTERPART_AREA'])
ser_oecd_partners = ser_oecd_partners.to_frame().join(ser_ison_countries).drop(0, axis = 1).squeeze()
for iter_iso_long in (ser_oecd_partners[ser_oecd_partners.isna()].index.get_level_values(0)):
    if iter_iso_long in ser_ison_countries.index:
        print('OECD Partner country with no ISON match:', iter_iso_long)
### ISON countries with no OECD partner match:
set_no_partners = set(ser_ison_countries.dropna().index) - set(ser_oecd_partners.index)
print('ISON countries with no OECD partner match:', set_no_partners, '(', len(set_no_partners), ')')

ISON countries with no OECD reporter match: {'NGA', 'BRA', 'ZAF', 'MAR', 'KWT', 'PER', 'VNM', 'THA', 'SAU', 'GHA', 'ARG', 'ECU', 'BGD', 'ROU', 'CIV', 'UGA', 'UKR', 'TWN', 'MLT', 'CYP', 'SRB', 'ZMB', 'JOR', 'IDN', 'KAZ', 'KEN', 'BWA', 'PAK', 'HKG', 'QAT', 'MUS', 'LBN', 'PAN', 'ARE', 'NAM', 'TUN', 'RUS', 'LKA', 'PHL', 'IND', 'MYS', 'OMN', 'BGR', 'CHN', 'HRV', 'SGP', 'EGY', 'BHR'} ( 48 )
ISON countries with no OECD partner match: {'ROU'} ( 1 )


In [15]:
### OECD FDI: FDI POSITION REQUEST CONSTRUCTING

str_fdi_pos_request_params = '.'.join(['', str_measure, str_direction, str_fdi_type, str_residence, str_accounting, str_counterpart, ''])
str_fdi_pos_request = str_oecd_base_url + str_fdi_pos_dataset_add + '/' + str_fdi_pos_request_params + '/all?startTime=' + str(date_start.year) + \
                      '&endTime=' + str(date_end.year) + '&detail=DataOnly'
obj_fdi_pos_dataset = request_session.get(str_fdi_pos_request).json()

In [16]:
### OECD FDI: FDI POSITION INDEX DATA COLLECTING:

### Dates:
list_idx_dates = []
for tup_date in obj_fdi_pos_dataset['structure']['dimensions']['observation'][0]['values']:
    list_idx_dates.append(pd.to_datetime(tup_date['id']) + pd.offsets.BYearEnd())
### Parameters:    
list_idx_library = []
for iter_position in obj_fdi_pos_dataset['structure']['dimensions']['series']:
    list_param_values = []
    for tup_parameter in iter_position['values']:
        list_param_values.append(tup_parameter['id'])            
    list_idx_library.append(list_param_values)
### Result:
list_idx_library.append(list_idx_dates)
### Converting to dictionary for future replacing:
list_idx_dict = []
for iter_list in list_idx_library:
    list_idx_dict.append(dict(zip(map(str, range(len(iter_list))), iter_list)))

In [17]:
### OECD FDI: FDI POSITION DATASET RESAMPLING

dict_datasets_res = {}
dict_datasets_source = obj_fdi_pos_dataset['dataSets'][0]['series']
### Parameters and date indexes integration:
for iter_dataset in dict_datasets_source:
    dict_observations = dict_datasets_source[iter_dataset]['observations']
    for iter_observation in dict_observations:
        str_iter_idx = iter_dataset + ':' + iter_observation
        flo_iter_value = dict_observations[iter_observation][0]
        dict_datasets_res[str_iter_idx] = flo_iter_value

In [18]:
### OECD FDI: FDI POSITION DATASET REINDEXATION

gc.collect()
df_fdi_pos_data = pd.Series(dict_datasets_res)
df_fdi_pos_data.index = pd.MultiIndex.from_arrays(zip(*df_fdi_pos_data.index.str.split(':')))
int_levels_number = df_fdi_pos_data.index.nlevels
df_fdi_pos_data = df_fdi_pos_data.reset_index()
### Replacing numbers with parameter values:
for iter_level in range(int_levels_number):
    df_fdi_pos_data['level_' + str(iter_level)].replace(list_idx_dict[iter_level], inplace = True)
    ### Replacing long ISO names with short ISO names:
    if (iter_level == 0):
        df_fdi_pos_data['level_' + str(iter_level)].replace(dict(zip(df_country_codes['ISO LONG'].values, df_country_codes['ISO SHORT'].values)), inplace = True)
    elif (iter_level == 7):
        df_fdi_pos_data['level_' + str(iter_level)].replace(dict(zip(df_country_codes['ISO LONG'].values, df_country_codes['ISO SHORT'].values)), inplace = True)
    ### Directions renaming:
    elif (iter_level == 2):
        df_fdi_pos_data['level_' + str(iter_level)].replace({'DI': 'Inward', 'DO': 'Outward'}, inplace = True)
    ### Investment types renaming:
    elif (iter_level == 3):
        df_fdi_pos_data['level_' + str(iter_level)].replace({'LE_FA_F': 'Total', 'LE_FA_F5': 'Equity'}, inplace = True)         
#    ### Flow types renaming:
#    elif (iter_level == 5):
#        df_fdi_pos_data['level_' + str(iter_level)].replace({'NET': 'Net', 'A': 'Asset', 'L': 'Liability'}, inplace = True)      

### Intergated observations dropping:
df_fdi_pos_data = df_fdi_pos_data.loc[
                                      df_fdi_pos_data['level_0'].isin(df_country_codes['ISO SHORT'].values) & 
                                      df_fdi_pos_data['level_7'].isin(df_country_codes['ISO SHORT'].values)
                                     ]
### Indexes defining:
ser_fdi_pos_data = df_fdi_pos_data.drop(['level_1', 'level_4', 'level_5', 'level_6'], axis = 1)\
                    .set_index(['level_3', 'level_2', 'level_8', 'level_0', 'level_7']).squeeze()
ser_fdi_pos_data.index.names = ['Type', 'Direction', 'Date', 'Reporter', 'Partner']
ser_fdi_pos_data.sort_index(inplace = True)
ser_fdi_pos_data = ser_fdi_pos_data[ser_fdi_pos_data.index.get_level_values('Reporter') != ser_fdi_pos_data.index.get_level_values('Partner')]
ser_fdi_pos_data[ser_fdi_pos_data < 0.0] = 0.0
ser_fdi_pos_data.name = 'FDI Positions'

In [19]:
### OECD FDI: FDI POSITION DATASETS SAVING

ser_fdi_pos_data.loc['Total', 'Outward', :, ser_ison_status.index, :].droplevel(['Type', 'Direction'])\
    .to_hdf(path_or_buf = str_path_oecd_fdi_dataset, key = str_key_do_total_oecd_fdi_dataset, mode = 'w', format = 'fixed')
ser_fdi_pos_data.loc['Equity', 'Outward', :, ser_ison_status.index, :].droplevel(['Type', 'Direction'])\
    .to_hdf(path_or_buf = str_path_oecd_fdi_dataset, key = str_key_do_equity_oecd_fdi_dataset, mode = 'a', format = 'fixed')
ser_total_di = ser_fdi_pos_data.loc['Total', 'Inward', :, :, ser_ison_status.index].droplevel(['Type', 'Direction'])
ser_total_di.index.names = ['Date', 'Partner', 'Reporter']
ser_total_di.reorder_levels(['Date', 'Reporter', 'Partner']).sort_index()\
    .to_hdf(path_or_buf = str_path_oecd_fdi_dataset, key = str_key_di_total_oecd_fdi_dataset, mode = 'a', format = 'fixed')
ser_equity_di = ser_fdi_pos_data.loc['Equity', 'Inward', :, :, ser_ison_status.index].droplevel(['Type', 'Direction'])
ser_equity_di.index.names = ['Date', 'Partner', 'Reporter']
ser_equity_di.reorder_levels(['Date', 'Reporter', 'Partner']).sort_index()\
    .to_hdf(path_or_buf = str_path_oecd_fdi_dataset, key = str_key_di_equity_oecd_fdi_dataset, mode = 'a', format = 'fixed')

In [7]:
### OECD FDI: TOTAL DATA AGGREGATION: DATASETS LOADING

gc.collect()
ser_oecd_asset = pd.read_hdf(path_or_buf = str_path_oecd_fdi_dataset, key = str_key_do_total_oecd_fdi_dataset)
ser_oecd_asset.name = 'Asset'
ser_oecd_liability_inv = pd.read_hdf(path_or_buf = str_path_oecd_fdi_dataset, key = str_key_di_total_oecd_fdi_dataset)
ser_oecd_liability_inv.name = 'Liability_Inverted'
df_oecd_total = pd.concat([ser_oecd_asset, ser_oecd_liability_inv], axis = 1, names = 'Data Source').astype('float32').round(2)

In [8]:
### OECD FDI: TOTAL DATA AGGREGATION: DATA QUALITY RATIOS

gc.collect()

### Defining similarity for investors by date
def get_investor_ratio(df_group):
#    df_group['Asset'].fillna(0.0, inplace = True)
    df_group.fillna(0.0, inplace = True)    
    df_both = df_group.dropna()    
    if (df_both['Asset'].sum() > 0.0):
        flo_result = (df_both['Asset'] - df_both['Liability_Inverted']).abs().clip(upper = df_group['Asset'].max()).sum() / df_group['Asset'].sum() / len(df_group)    
    else:
        flo_result = np.NaN    
    return flo_result
### Defining similarity for borrowers by date
def get_borrower_ratio(df_group):
#    df_group['Liability_Inverted'].fillna(0.0, inplace = True)
    df_group.fillna(0.0, inplace = True)     
    df_both = df_group.dropna()
    if (df_both['Liability_Inverted'].sum() > 0.0):
        flo_result = (df_both['Asset'] - df_both['Liability_Inverted']).abs().clip(upper = df_group['Liability_Inverted'].max()).sum() \
                                                                        / df_group['Liability_Inverted'].sum() / len(df_group)
    else:
        flo_result = np.NaN    
    return flo_result
### Similarity values calculation:
ser_investor_ratio = df_oecd_total.groupby(['Date', 'Reporter']).apply(get_investor_ratio)
ser_investor_ratio.name = 'Investor_Ratio'
ser_borrower_ratio = df_oecd_total.groupby(['Date', 'Partner']).apply(get_borrower_ratio)
ser_borrower_ratio.name = 'Borrower_Ratio'

In [9]:
### OECD FDI: TOTAL DATA AGGREGATION: SIMILARITY TEST

print(round(ser_borrower_ratio.min(), 4), '/', ser_borrower_ratio.idxmin())
print(round(ser_borrower_ratio.max(), 4), '/', ser_borrower_ratio.idxmax())

#display(df_oecd_total.loc[('2019-12-31', All, 'ES'), :])
#display(df_oecd_total.loc[('2017-12-29', All, 'IL'), :].dropna())

0.0025 / (Timestamp('2019-12-31 00:00:00'), 'ES')
0.0406 / (Timestamp('2017-12-29 00:00:00'), 'IL')


In [10]:
### OECD FDI: TOTAL DATA AGGREGATION: SIMILARITY TEST

print(round(ser_investor_ratio.min(), 4), '/', ser_investor_ratio.idxmin())
print(round(ser_investor_ratio.max(), 4), '/', ser_investor_ratio.idxmax())

#display(df_oecd_total.loc[('2015-12-31', 'FR', All), :])
#display(df_oecd_total.loc[('2014-12-31', 'NZ', All), :].dropna())

0.0013 / (Timestamp('2015-12-31 00:00:00'), 'FR')
0.0171 / (Timestamp('2014-12-31 00:00:00'), 'NZ')


In [11]:
### OECD FDI: TOTAL DATA AGGREGATION: ADDING RATIOS

df_oecd_to_augment = df_oecd_total.join(ser_investor_ratio).join(ser_borrower_ratio)
df_oecd_to_augment['Asset_Augmented'] = np.NaN # -999 # 
#df_cpis_augmented['Verified'] = False
df_oecd_to_augment = df_oecd_to_augment.reorder_levels(['Date', 'Reporter', 'Partner'])

In [12]:
### OECD FDI: TOTAL DATA AGGREGATION: CONDITIONAL REPLACING

gc.collect()
def augment_by_date(df_date, int_option = -1):
    '''
       -1 : Replace NaN Asset values unconditionally
        0 : Replace NaN Asset values when Investor's Ratio > Borrower's Ratio
        1 : Replace NaN or zero Asset values when Investor's Ratio > Borrower's Ratio
        2 : Replace any Asset values when Investor's Ratio > Borrower's Ratio
    '''
    if (int_option == -1):
        ### Replacing zero Asset & Liability values with NaN:
        df_date.loc[df_date['Asset'] == 0.0, 'Asset'] = np.NaN        
        df_date['Asset_Augmented'] = df_date['Asset'].combine_first(df_date['Liability_Inverted'])
    elif (int_option == 0):
        ### Fill resulting column with not NaN Asset values:
        df_date.loc[df_date['Asset'].notna(), 'Asset_Augmented'] = df_date[df_date['Asset'].notna()]['Asset'].values
        ### Fill resulting column with Liability value if Asset value is NaN & Investor Ratio is NaN (and doesn't matter if Borrower Ratio is NaN):
        df_date.loc[df_date['Asset'].isna() & df_date['Investor_Ratio'].isna(), 'Asset_Augmented'] = \
            df_date[df_date['Asset'].isna() & df_date['Investor_Ratio'].isna()]['Liability_Inverted'].values
        ### Fill resulting column with Liability value if Asset value is NaN & Investor Ratio is bigger than Borrower Ratio:
        df_date.loc[df_date['Asset'].isna() & df_date['Investor_Ratio'].notna() & df_date['Borrower_Ratio'].notna() & \
                    (df_date['Investor_Ratio'] > df_date['Borrower_Ratio']), 'Asset_Augmented'] = \
            df_date[df_date['Asset'].isna() & df_date['Investor_Ratio'].notna() & df_date['Borrower_Ratio'].notna() & \
                    (df_date['Investor_Ratio'] > df_date['Borrower_Ratio'])]['Liability_Inverted'].values
    elif (int_option == 1):
        ### Replacing zero Asset values with NaN:
        df_date.loc[df_date['Asset'] == 0.0, 'Asset'] = np.NaN
        ### Fill resulting column with not NaN Asset values:
        df_date.loc[df_date['Asset'].notna(), 'Asset_Augmented'] = df_date.loc[df_date['Asset'].notna(), 'Asset'].values
        ### Fill resulting column with Liability value if Asset value is NaN & Investor Ratio is NaN (and doesn't matter if Borrower Ratio is NaN):
        df_date.loc[df_date['Asset'].isna() & df_date['Investor_Ratio'].isna(), 'Asset_Augmented'] = \
            df_date[df_date['Asset'].isna() & df_date['Investor_Ratio'].isna()]['Liability_Inverted'].values
        ### Fill resulting column with Liability value if Asset value is NaN & Investor Ratio is bigger than Borrower Ratio:
        df_date.loc[df_date['Asset'].isna() & df_date['Investor_Ratio'].notna() & df_date['Borrower_Ratio'].notna() & \
                    (df_date['Investor_Ratio'] > df_date['Borrower_Ratio']), 'Asset_Augmented'] = \
            df_date[df_date['Asset'].isna() & df_date['Investor_Ratio'].notna() & df_date['Borrower_Ratio'].notna() & \
                    (df_date['Investor_Ratio'] > df_date['Borrower_Ratio'])]['Liability_Inverted'].values
    else:
        ### Replacing zero Asset & Liability values with NaN:
        df_date.loc[df_date['Asset'] == 0.0, 'Asset'] = np.NaN        
        df_date.loc[df_date['Liability_Inverted'] == 0.0, 'Liability_Inverted'] = np.NaN
        ### Ratios preparation:
        df_date.loc[df_date['Investor_Ratio'].isna(), 'Investor_Ratio'] = 999.0
        df_date.loc[df_date['Borrower_Ratio'].isna(), 'Borrower_Ratio'] = 1000.0
        ### Ratios comparision:
        df_date.loc[df_date['Investor_Ratio'] <= df_date['Borrower_Ratio'], 'Asset_Augmented'] = \
            df_date[df_date['Investor_Ratio'] <= df_date['Borrower_Ratio']]['Asset'].values
        df_date.loc[df_date['Investor_Ratio'] > df_date['Borrower_Ratio'], 'Asset_Augmented'] = \
            df_date[df_date['Investor_Ratio'] > df_date['Borrower_Ratio']]['Liability_Inverted'].values                                       
    return df_date

dict_oecd_augmented = {}
dict_oecd_augmented[-1] = df_oecd_to_augment.groupby('Date').apply(augment_by_date, -1)
dict_oecd_augmented[0] = df_oecd_to_augment.groupby('Date').apply(augment_by_date, 0)
dict_oecd_augmented[1] = df_oecd_to_augment.groupby('Date').apply(augment_by_date, 1)
dict_oecd_augmented[2] = df_oecd_to_augment.groupby('Date').apply(augment_by_date, 2)

In [69]:
### OECD FDI: TOTAL DATA AGGREGATION: RESULTS TESTING

ser_quantity = df_oecd_to_augment['Asset'].replace({0.0: np.NaN}).groupby(['Date','Reporter']).count()
ser_quantity = ser_quantity[ser_quantity > 0].groupby('Date').count()
ser_quantity.name = 'raw'
ser_quantity.to_excel('Data_Files/Test_Files/Augmentation_Test_' + ser_quantity.name + '.xlsx', merge_cells = False)

for iter_option in dict_oecd_augmented:
    dt_option = dict_oecd_augmented[iter_option]
    ser_quantity = dt_option['Asset_Augmented'].replace({0.0: np.NaN}).groupby(['Date','Reporter']).count()
    ser_quantity = ser_quantity[ser_quantity > 0].groupby('Date').count()
    ser_quantity.name = str(iter_option)
    ser_quantity.to_excel('Data_Files/Test_Files/Augmentation_Test_' + str(iter_option) + '.xlsx', merge_cells = False)

In [82]:
### OECD FDI: TOTAL DATA AGGREGATION: RESULTS SAVING TO SERIES

ser_total_augmented = dict_oecd_augmented[2]['Asset_Augmented']
ser_total_augmented.name = 'Total'
ser_total_augmented.replace({0.0: np.NaN})\
    .to_hdf(path_or_buf = str_path_oecd_fdi_augmented, key = str_key_do_total_oecd_fdi_augmented, mode = 'w', format = 'fixed')

In [13]:
### OECD FDI: TOTAL DATA AGGREGATION: RESULTS CONSOLIDATION TO DATAFRAME AND SAVING

df_augmentation_way = pd.concat([df_oecd_to_augment['Asset'].replace({0.0: np.NaN}), 
                                 dict_oecd_augmented[-1]['Asset_Augmented'], 
                                 dict_oecd_augmented[2]['Asset_Augmented']], 
                                axis = 1, keys = ['Assets_Only', 'Unconditional', 'Option_2'], names = 'Augmentation_Way')
df_augmentation_way.to_hdf(path_or_buf = str_path_oecd_fdi_options, key = str_key_total_oecd_fdi_options, mode = 'w', format = 'fixed')

In [15]:
### OECD FDI: EQUITY DATA AGGREGATION: DATASETS LOADING

gc.collect()
ser_oecd_asset = pd.read_hdf(path_or_buf = str_path_oecd_fdi_dataset, key = str_key_do_equity_oecd_fdi_dataset)
ser_oecd_asset.name = 'Asset'
ser_oecd_liability_inv = pd.read_hdf(path_or_buf = str_path_oecd_fdi_dataset, key = str_key_di_equity_oecd_fdi_dataset)
ser_oecd_liability_inv.name = 'Liability_Inverted'
df_oecd_equity = pd.concat([ser_oecd_asset, ser_oecd_liability_inv], axis = 1, names = 'Data Source').astype('float32').round(2)

In [16]:
### OECD FDI: EQUITY DATA AGGREGATION: DATA QUALITY RATIOS

gc.collect()

### Defining similarity for investors by date
def get_investor_ratio(df_group):
#    df_group['Asset'].fillna(0.0, inplace = True)
    df_group.fillna(0.0, inplace = True)    
    df_both = df_group.dropna()    
    if (df_both['Asset'].sum() > 0.0):
        flo_result = (df_both['Asset'] - df_both['Liability_Inverted']).abs().clip(upper = df_group['Asset'].max()).sum() / df_group['Asset'].sum() / len(df_group)    
    else:
        flo_result = np.NaN    
    return flo_result
### Defining similarity for borrowers by date
def get_borrower_ratio(df_group):
#    df_group['Liability_Inverted'].fillna(0.0, inplace = True)
    df_group.fillna(0.0, inplace = True)     
    df_both = df_group.dropna()
    if (df_both['Liability_Inverted'].sum() > 0.0):
        flo_result = (df_both['Asset'] - df_both['Liability_Inverted']).abs().clip(upper = df_group['Liability_Inverted'].max()).sum() \
                                                                        / df_group['Liability_Inverted'].sum() / len(df_group)
    else:
        flo_result = np.NaN    
    return flo_result
### Similarity values calculation:
ser_investor_ratio = df_oecd_equity.groupby(['Date', 'Reporter']).apply(get_investor_ratio)
ser_investor_ratio.name = 'Investor_Ratio'
ser_borrower_ratio = df_oecd_equity.groupby(['Date', 'Partner']).apply(get_borrower_ratio)
ser_borrower_ratio.name = 'Borrower_Ratio'

In [17]:
### OECD FDI: EQUITY DATA AGGREGATION: SIMILARITY TEST

print(round(ser_borrower_ratio.min(), 4), '/', ser_borrower_ratio.idxmin())
print(round(ser_borrower_ratio.max(), 4), '/', ser_borrower_ratio.idxmax())

#display(df_oecd_equity.loc[('2020-12-31', All, 'IT'), :].dropna())
#display(df_oecd_equity.loc[('2017-12-29', All, 'IL'), :].dropna())

0.0026 / (Timestamp('2020-12-31 00:00:00'), 'IT')
0.0526 / (Timestamp('2017-12-29 00:00:00'), 'IL')


In [18]:
### OECD FDI: EQUITY DATA AGGREGATION: SIMILARITY TEST

print(round(ser_investor_ratio.min(), 4), '/', ser_investor_ratio.idxmin())
print(round(ser_investor_ratio.max(), 4), '/', ser_investor_ratio.idxmax())

#display(df_oecd_equity.loc[('2019-12-31', 'BE', All), :].dropna())
#display(df_oecd_equity.loc[('2016-12-30', 'NZ', All), :].dropna())

0.0012 / (Timestamp('2019-12-31 00:00:00'), 'BE')
0.0192 / (Timestamp('2016-12-30 00:00:00'), 'NZ')


In [19]:
### OECD FDI: EQUITY DATA AGGREGATION: ADDING RATIOS

df_oecd_to_augment = df_oecd_equity.join(ser_investor_ratio).join(ser_borrower_ratio)
df_oecd_to_augment['Asset_Augmented'] = np.NaN # -999 # 
#df_cpis_augmented['Verified'] = False
df_oecd_to_augment = df_oecd_to_augment.reorder_levels(['Date', 'Reporter', 'Partner'])

In [20]:
### OECD FDI: EQUITY DATA AGGREGATION: CONDITIONAL REPLACING

gc.collect()
def augment_by_date(df_date, int_option = -1):
    '''
       -1 : Replace NaN Asset values unconditionally
        0 : Replace NaN Asset values when Investor's Ratio > Borrower's Ratio
        1 : Replace NaN or zero Asset values when Investor's Ratio > Borrower's Ratio
        2 : Replace any Asset values when Investor's Ratio > Borrower's Ratio
    '''
    if (int_option == -1):
        ### Replacing zero Asset & Liability values with NaN:
        df_date.loc[df_date['Asset'] == 0.0, 'Asset'] = np.NaN        
        df_date['Asset_Augmented'] = df_date['Asset'].combine_first(df_date['Liability_Inverted'])
    elif (int_option == 0):
        ### Fill resulting column with not NaN Asset values:
        df_date.loc[df_date['Asset'].notna(), 'Asset_Augmented'] = df_date[df_date['Asset'].notna()]['Asset'].values
        ### Fill resulting column with Liability value if Asset value is NaN & Investor Ratio is NaN (and doesn't matter if Borrower Ratio is NaN):
        df_date.loc[df_date['Asset'].isna() & df_date['Investor_Ratio'].isna(), 'Asset_Augmented'] = \
            df_date[df_date['Asset'].isna() & df_date['Investor_Ratio'].isna()]['Liability_Inverted'].values
        ### Fill resulting column with Liability value if Asset value is NaN & Investor Ratio is bigger than Borrower Ratio:
        df_date.loc[df_date['Asset'].isna() & df_date['Investor_Ratio'].notna() & df_date['Borrower_Ratio'].notna() & \
                    (df_date['Investor_Ratio'] > df_date['Borrower_Ratio']), 'Asset_Augmented'] = \
            df_date[df_date['Asset'].isna() & df_date['Investor_Ratio'].notna() & df_date['Borrower_Ratio'].notna() & \
                    (df_date['Investor_Ratio'] > df_date['Borrower_Ratio'])]['Liability_Inverted'].values
    elif (int_option == 1):
        ### Replacing zero Asset values with NaN:
        df_date.loc[df_date['Asset'] == 0.0, 'Asset'] = np.NaN
        ### Fill resulting column with not NaN Asset values:
        df_date.loc[df_date['Asset'].notna(), 'Asset_Augmented'] = df_date.loc[df_date['Asset'].notna(), 'Asset'].values
        ### Fill resulting column with Liability value if Asset value is NaN & Investor Ratio is NaN (and doesn't matter if Borrower Ratio is NaN):
        df_date.loc[df_date['Asset'].isna() & df_date['Investor_Ratio'].isna(), 'Asset_Augmented'] = \
            df_date[df_date['Asset'].isna() & df_date['Investor_Ratio'].isna()]['Liability_Inverted'].values
        ### Fill resulting column with Liability value if Asset value is NaN & Investor Ratio is bigger than Borrower Ratio:
        df_date.loc[df_date['Asset'].isna() & df_date['Investor_Ratio'].notna() & df_date['Borrower_Ratio'].notna() & \
                    (df_date['Investor_Ratio'] > df_date['Borrower_Ratio']), 'Asset_Augmented'] = \
            df_date[df_date['Asset'].isna() & df_date['Investor_Ratio'].notna() & df_date['Borrower_Ratio'].notna() & \
                    (df_date['Investor_Ratio'] > df_date['Borrower_Ratio'])]['Liability_Inverted'].values
    else:
        ### Replacing zero Asset & Liability values with NaN:
        df_date.loc[df_date['Asset'] == 0.0, 'Asset'] = np.NaN        
        df_date.loc[df_date['Liability_Inverted'] == 0.0, 'Liability_Inverted'] = np.NaN
        ### Ratios preparation:
        df_date.loc[df_date['Investor_Ratio'].isna(), 'Investor_Ratio'] = 999.0
        df_date.loc[df_date['Borrower_Ratio'].isna(), 'Borrower_Ratio'] = 1000.0
        ### Ratios comparision:
        df_date.loc[df_date['Investor_Ratio'] <= df_date['Borrower_Ratio'], 'Asset_Augmented'] = \
            df_date[df_date['Investor_Ratio'] <= df_date['Borrower_Ratio']]['Asset'].values
        df_date.loc[df_date['Investor_Ratio'] > df_date['Borrower_Ratio'], 'Asset_Augmented'] = \
            df_date[df_date['Investor_Ratio'] > df_date['Borrower_Ratio']]['Liability_Inverted'].values                                       
    return df_date

dict_oecd_augmented = {}
dict_oecd_augmented[-1] = df_oecd_to_augment.groupby('Date').apply(augment_by_date, -1)
dict_oecd_augmented[0] = df_oecd_to_augment.groupby('Date').apply(augment_by_date, 0)
dict_oecd_augmented[1] = df_oecd_to_augment.groupby('Date').apply(augment_by_date, 1)
dict_oecd_augmented[2] = df_oecd_to_augment.groupby('Date').apply(augment_by_date, 2)

In [18]:
### OECD FDI: EQUITY DATA AGGREGATION: RESULTS TESTING

ser_quantity = df_oecd_to_augment['Asset'].replace({0.0: np.NaN}).groupby(['Date','Reporter']).count()
ser_quantity = ser_quantity[ser_quantity > 0].groupby('Date').count()
ser_quantity.name = 'raw'
ser_quantity.to_excel('Data_Files/Test_Files/Augmentation_Test_' + ser_quantity.name + '.xlsx', merge_cells = False)

for iter_option in dict_oecd_augmented:
    dt_option = dict_oecd_augmented[iter_option]
    ser_quantity = dt_option['Asset_Augmented'].replace({0.0: np.NaN}).groupby(['Date','Reporter']).count()
    ser_quantity = ser_quantity[ser_quantity > 0].groupby('Date').count()
    ser_quantity.name = str(iter_option)
    ser_quantity.to_excel('Data_Files/Test_Files/Augmentation_Test_' + str(iter_option) + '.xlsx', merge_cells = False)

In [33]:
### OECD FDI: EQUITY DATA AGGREGATION: RESULTS SAVING (SERIES)

ser_equity_augmented = dict_oecd_augmented[2]['Asset_Augmented']
ser_equity_augmented.name = 'Total'
ser_equity_augmented.replace({0.0: np.NaN})\
    .to_hdf(path_or_buf = str_path_oecd_fdi_augmented, key = str_key_do_equity_oecd_fdi_augmented, mode = 'a', format = 'fixed')

In [21]:
### OECD FDI: EQUITY DATA AGGREGATION: RESULTS CONSOLIDATION TO DATAFRAME AND SAVING

df_augmentation_way = pd.concat([df_oecd_to_augment['Asset'].replace({0.0: np.NaN}), 
                                 dict_oecd_augmented[-1]['Asset_Augmented'], 
                                 dict_oecd_augmented[2]['Asset_Augmented']], 
                                axis = 1, keys = ['Assets_Only', 'Unconditional', 'Option_2'], names = 'Augmentation_Way')
df_augmentation_way.to_hdf(path_or_buf = str_path_oecd_fdi_options, key = str_key_equity_oecd_fdi_options, mode = 'a', format = 'fixed')